# Kubernetes

The following note is for personal purposes for understanding the architecture of [kubernetes](https://kubernetes.io/) with the resources from [edx](https://learning.edx.org/course/course-v1:LinuxFoundationX+LFS158x+1T2022/block-v1:LinuxFoundationX+LFS158x+1T2022+type@sequential+block@c77f2d1b11234afb9886680612041ab2/block-v1:LinuxFoundationX+LFS158x+1T2022+type@vertical+block@6893f0526bfe4a8c81ae62633aa6f06b).

Legacy softwares that use assembly codes are highly vulnerable to downtime when it comes to updating features and developing new features. Such applications are **Monolithic** and very inefficient, as opposed to modern architecture that has functions broken down into microservices or **Pebbles**. 

**Index**
- []()


### Container

Before learning about kubernetes, it is important to understand the key concepts of what a [Container](https://learning.edx.org/course/course-v1:LinuxFoundationX+LFS158x+1T2022/block-v1:LinuxFoundationX+LFS158x+1T2022+type@sequential+block@c77f2d1b11234afb9886680612041ab2/block-v1:LinuxFoundationX+LFS158x+1T2022+type@vertical+block@453babddfd324944b4f6623bd18f3536) is.

**Containers** are an *application-centric method to deliver high-performing, scalable applications on any infrastructure of choice*. Containers are best suited to deliver microservices by providing portable isolated virtual environments for applications to run without interference from other running applications. 

![pic](./pics/containers.png)

__Microservices__ are lightweight applications written within various modern programming languages, with specific dependencies, libraries and envrionmental requirements.

***Container*** does not run the apps directly, but the **Container Images**. A **Container Image** bundles the application along with its runtime, libraries and dependencies. 


### Container Orchestration

It would be unwise to run all the microservices in a single **Container**, because it would be counterintuitive to ***Container Orchestration***